# Analyse results of benchmark

And because i love it, we will do it using SQL & duckdb, yeah!

In [2]:
import duckdb

In [3]:
duckdb.sql("""
SELECT 
    function,
    avg(time_in_s)  as avg_time_in_s,
    avg(memory_in_mb) as avg_memory_in_mb
FROM read_json('data/benchmark_results.json')
GROUP BY 1
ORDER BY avg_time_in_s 
""")

┌───────────────────┬────────────────────┬────────────────────┐
│     function      │   avg_time_in_s    │  avg_memory_in_mb  │
│      varchar      │       double       │       double       │
├───────────────────┼────────────────────┼────────────────────┤
│ manual_flatten    │        0.026066467 │         18.6657568 │
│ flatdict_flatten  │       0.0915694166 │          0.1801168 │
│ generator_flatten │       0.3713328834 │ 17.252613399999998 │
│ pandas_flatten    │ 2.8101268414000002 │         81.4832046 │
│ dlt_flatten       │        4.793875925 │           0.974562 │
└───────────────────┴────────────────────┴────────────────────┘

In [4]:
duckdb.sql("""
SELECT 
    function,
    avg(time_in_s)  as avg_time_in_s,
    avg(memory_in_mb) as avg_memory_in_mb
FROM read_json('data/benchmark_results.json')
GROUP BY 1
ORDER BY avg_memory_in_mb 
""")

┌───────────────────┬────────────────────┬────────────────────┐
│     function      │   avg_time_in_s    │  avg_memory_in_mb  │
│      varchar      │       double       │       double       │
├───────────────────┼────────────────────┼────────────────────┤
│ flatdict_flatten  │       0.0915694166 │          0.1801168 │
│ dlt_flatten       │        4.793875925 │           0.974562 │
│ generator_flatten │       0.3713328834 │ 17.252613399999998 │
│ manual_flatten    │        0.026066467 │         18.6657568 │
│ pandas_flatten    │ 2.8101268414000002 │         81.4832046 │
└───────────────────┴────────────────────┴────────────────────┘

In [10]:
duckdb.sql("""
SELECT 
    num_players,
    function,
    min(time_in_s) OVER(PARTITION BY num_players ORDER BY time_in_s ) as min_time,
    max(time_in_s) OVER(PARTITION BY num_players ORDER BY time_in_s ) as max_time,
    avg(time_in_s) OVER(PARTITION BY num_players ORDER BY time_in_s ) as avg_time_in_s,
FROM read_json('data/benchmark_results.json')
ORDER BY avg_time_in_s;
""")

┌─────────────┬───────────────────┬─────────────┬──────────────┬────────────────────────┐
│ num_players │     function      │  min_time   │   max_time   │     avg_time_in_s      │
│    int64    │      varchar      │   double    │    double    │         double         │
├─────────────┼───────────────────┼─────────────┼──────────────┼────────────────────────┤
│          23 │ manual_flatten    │  2.1334e-05 │   2.1334e-05 │             2.1334e-05 │
│          23 │ flatdict_flatten  │  2.1334e-05 │   0.00010425 │             6.2792e-05 │
│         100 │ manual_flatten    │  9.3084e-05 │   9.3084e-05 │             9.3084e-05 │
│          23 │ generator_flatten │  2.1334e-05 │  0.000502708 │ 0.00020943066666666667 │
│         100 │ flatdict_flatten  │  9.3084e-05 │  0.000417708 │ 0.00025539599999999997 │
│         100 │ generator_flatten │  9.3084e-05 │   0.00169725 │            0.000736014 │
│        1000 │ manual_flatten    │ 0.001213833 │  0.001213833 │            0.001213833 │
│         